## 7.7 在匿名函数中绑定变量的值
### 7.7.1 问题
利用`lambda`表达式定义了一个匿名函数，但是也希望可以在函数定义的时候完成对特殊变量的绑定。
### 7.7.2 解决方案
考虑下列代码的行为，这里的问题在于`lambda`表达式中用到的x是一个自由变量，在运行时才进行绑定而不是定义的时候绑定，因此，`lambda`表达式中x的值应该在执行时确定的，执行时x的值是多少就是多少。

In [2]:
x = 10
a = lambda y: x + y
x = 20
b = lambda y: x + y

In [3]:
print(a(10))   # 30
print(b(10))   # 30

30
30


In [4]:
x = 15
print(a(10))    # 25
x = 3
print(a(10))    # 13

25
13


如果希望匿名函数可以在定义的时候绑定变量，并保持值不变，可以将那个值作为默认参数实现。

In [5]:
x = 10
a = lambda y, x=x: x + y
x = 20
b = lambda y, x=x: x + y

In [6]:
print(a(10))   # 20
print(b(10))   # 30

20
30


### 7.7.3 讨论
本节提到的问题一般比较容易出现在那些对`lambda`函数过于“聪明”的应用上。比方说，通过列表推导式创建一列`lambda`表达式，或者在一个循环中期望`lambda`表达式能够在定义的时候记住迭代变量。示例如下：

In [7]:
func = [lambda x: x+n for n in range(5)]
for f in func:      # 运行时n最后的值为4，所以调用f(0)时值均为4
    print(f(0))

4
4
4
4
4


注意到所有的函数都认为n的值为4，也就是迭代中的最后一个值。和下面的代码做下对比，可以看到，现在函数可以在定义的时候捕获到n的值了。

In [8]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4


## 7.8 让带有N个参数的可调用对象以较少的参数形式调用
### 7.8.1 问题
有一个可调用对象可能会以回调函数的形式同其他的Python代码交互。但是这个可调用对象需要的参数过多，如果直接调用的话会产生异常。
### 7.8.2 解决方案
如果需要减少函数的参数数量，应该使用`functools.partial()`。函数`partial()`允许我们给一个或多个参数指定固定的值，以此减少需要提供给之后调用的参数数量。为了说明这个过程，假设有这么一个函数：

In [9]:
def spam(a, b, c, d):
    print(a, b, c, d)

from functools import partial
s1 = partial(spam, 1)   # a = 1
s1(2, 3, 4)

1 2 3 4


In [10]:
s2 = partial(spam, d=42)
s2(1, 2, 3)

1 2 3 42


In [11]:
s2(4, 5, 5)

4 5 5 42


In [12]:
s3 = partial(spam, 1, 2, d=42)  # a=1, b=2, d=42
s3(100)

1 2 100 42


可以观察到`partial()`对特定的参数赋了固定值并返回了一个全新的可调用对象。这个新的可调用对象仍然需要通过指定那些未被赋值的参数来调用。这个新的可调用对象将传递给`partial()`的固定参数结合起来，统一将所有的参数传递给原始的函数。
### 7.8.3 讨论
本节提到的技术对于将看似不兼容的代码结合起来使用是大有裨益的。

第一个例子是，假设有一列以元组(x, y)来表示的点坐标。可以用下面的函数来计算两点之间的距离.

现在假设项根据这些点之间的距离来对它们排序。列表的`sort()`方法可接受一个key参数，它可用来做自定义的排序处理。但是它只能和接受单参数的函数一起工作(因此和distance()是不兼容的)。可以用`partial()`来解决这个问题：

In [13]:
points = [(1, 2), (3, 4), (5, 6), (7, 8)]
import math

def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2-x1, y2-y1)

pt = (4, 3)
points.sort(key=partial(distance, pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

可以对这个思路进行扩展，`partial()`常常可用来调整其他库用到的回调函数的参数签名。比方说，这里有一段代码利用`multiprocessing`模块以异步方式计算某个结果，并将这个结果传递给一个回调函数。该回调函数可接受这个结果以及一个可选的日志参数：

在`apply_async()`中指定回调函数时，额外的日志参数是通过`partial()`来指定的。`multiprocessing`模块对于这些细节一无所知——它只通过单个参数来调用回调函数。

In [14]:
def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)

# A sample function
def add(x, y):
    return x + y

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    from functools import partial

    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')

    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join

## 7.9 用函数替代只有单个方法的类
### 7.9.1 问题
有一个只定义了一个方法的类(除__init__()方法外)。但是，为了简化代码，更希望能够只用一个简单的函数来替代。
### 7.9.2 解决方案
在许多情况下，只有单个方法的类可以通过闭包(closure)将其转换成函数。考虑下面这个例子，这个类允许用户通过某种模板方案来获取URL。

In [ ]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

# Example use. Download stock data from yahoo
yahoo = UrlTemplate('http://finance.yahoo.com/d/quates.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

# 这个类可以用一个简单的函数来取代：
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# Example use
yahoo = urltemplate('http://finance.yahoo.com/d/quates.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

### 7.9.3 讨论
在许多情况下，我们会**使用只有单个方法的类的唯一原因就是保存额外的状态给类方法使用**。比方说，UelTemplate类的唯一目的就是将template的值保存在某处，这样就可以在`open()`方法中用上它。

按照我们给出的解决方案，使用嵌套函数或者说闭包常常会显得更加优雅。简单来说，**闭包就是一个函数，但是它保存着额外的变量环境，使得这些变量可以在函数中使用**。闭包的核心特性就是它可以记住定义闭包时的环境。因此，在这个解决方案中`open()`函数可以记住参数template的值，然后在随后的调用中使用该值。

当在编写代码中遇到**需要附加额外的状态给函数时，可以考虑使用闭包**。

## 7.10 在回调函数中携带额外的状态
### 7.10.1 问题
编写需要使用回调函数的代码(比如，时间处理例程、完成回调等)，但是希望回调函数可以携带额外的状态以便在回调函数内部使用。
### 7.10.2 解决方案
本节中提到的对回调函数的应用可以在许多库和框架中找到——尤其是那些和异步处理相关的库和框架。为了说明和测试的目的，首席按定义下面的函数，特它会调用一个回调函数：

In [16]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)

# 下面示例展示上述代码使用：
def print_result(result):
    print('Got', result)

def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)

Got 5


In [17]:
apply_async(add, ('hello', 'world'), callback=print_result)

Got helloworld


注意到函数`print_result()`仅接受一个单独的参数，也就是result。这里并没有传入其他的信息到函数中，有时候当我们希望回调函数可以同其他变量或者部分环境进行交互时，缺乏这类信息就会带来问题。

一种在回调函数中携带额外信息的方法是使用绑定方法(bound-method)而不是普通的函数。比如，下面这个类保存了一个内部的序列号码，每当接收到一个结果时就递增这个号码。

In [18]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

# 要使用这个类，可以创建一个实例并将绑定方法handler当做回调函数来用：
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ('hello', 'world'), callback=r.handler)

[1] Got: 5
[2] Got: helloworld


In [19]:
# 作为类的替代方案，可以使用闭包来捕获状态
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

[1] Got: 5
[2] Got: helloworld


In [20]:
# 利用协程(coroutine)来完成同样的任务，使用它的send()方法作为回调函数：
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

handler = make_handler()
next(handler)
apply_async(add, (2, 3), callback=handler.send)
apply_async(add, ('hello', 'world'), callback=handler.send)

[1] Got: 5
[2] Got: helloworld


In [24]:
# 也可以通过额外的参数在回调函数中携带状态，然后用partial()来处理参数个数的问题
class SequenceNo:
    def __init__(self):
        self.sequence = 0


def handler(result, seq):
    seq.sequence += 1
    print('[{}] Got: {}'.format(seq.sequence, result))

seq = SequenceNo()
from functools import partial
apply_async(add, (2, 3), callback=partial(handler, seq=seq))
apply_async(add, ('hello', 'world'), callback=partial(handler, seq=seq))

apply_async(add, (2, 3), callback=lambda r:handler(r, seq))

[1] Got: 5
[2] Got: helloworld
[3] Got: 5


### 7.10.3 讨论
基于回调函数的软件涉及常常会面临使代码陷入一团乱码的风险。部分原因是因为从代码发出初始请求开始到回调执行的这个过程，回调函数常常是与这个环境脱离的。因此，在发出请求和处理结果之间的执行环境就丢失了。如果想让回调函数在涉及多个步骤的任务中能够继续执行，就必须清除应该如何保存和还原相关的状态。

主要有两种方法可用于捕获和携带状态。可以在类实例上携带状态(将状态附加到绑定方法上)，也可以在闭包中携带状态。这两种方法中，闭包可能要显得更轻量级以下，而且由于闭包也是由函数构建的，这样显得会更加自然。

## 7.11 内联回调函数
### 7.11.1 问题
编写使用回调函数的代码，但是担心小型函数在代码中大肆泛滥，程序控制会因此而失控。
### 7.12.2 解决方案
可以通过生成器和写成将回调函数内联到一个函数中。


In [37]:
def aaply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)

from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

In [38]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [39]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


### 7.11.3 讨论
本节将真正考验以下读者对回调函数、生成器以及程序控制流方面的掌控情况。

首先，子啊涉及回调函数的代码中，问题的关键就在于当前的计算会被挂起，然后再稍后某个时刻再得到恢复。当计算得到恢复时，会带哦函数将得以继续处理执行。示例中的`apply_async()`函数对执行回调函数的关键部分做了简单的说明，尽管再显示世界中这回复杂得多(涉及线程、进程、时间处理例程等)。

将计算挂起之后再恢复，这个思想非常自然地同生成器函数对应了起来。棘突按理说就是`yield`操作使得生成器函数产生出一个值然后就挂起，后续调用生成器地`__next__()`或者`send()`方法会使得它再次启动。

鉴于此，本节的核心就在`inlned_async()`装饰器函数中。关键点就是对于生成器函数的所有yield语句装饰器都会逐条进行跟踪，以此一个。为了做到这点，我们创建了一个队列用来保存结果，初始时用None来填充。之后通过循环将结果从队列总取出，然后发送给生成器，这样就会产生下一次yield，此时就会接收到Async的实例。然后在循环中查找函数和参数，开始异步计算`apply_async()`。但是，这个过程中最为隐蔽的部分就在于这里没有使用普通的回调函数，回调过程被设定到队列的`put()`方法中。

此时应该可以精确描述到底发生了些什么。主循环会迅速回到顶层，并在队列中执行一个`get()`操作。如果有数据存在，那它就一定是由`put()`回调产生的结果。如果什么都没有，操作就会阻塞，等待之后某个时刻会有结果到来。至于结果要如何产生，这却决于`apply_async()`函数的实现。

如果对这些东西能否正常工作抱有怀疑，可以结合多进程库异步操作在单独的进程中执行，以此测试该方案：

In [40]:
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()
    apply_async = pool.apply_async

    # Run the test function
    test()

将精巧的控制流隐藏在生成器函数之后，这种做法可以在标准库以及第三方报中找到，比如说，`contextlib`模块中的`@contextmanager`装饰器也使用了类似的令人费解的技巧，将上下文管理器的入口点和出口点通过一个yield语句粘合在一起。 

## 7.12 访问定义在闭包内的变量
### 7.12.1 问题
希望通过函数来扩展闭包，使得在闭包内层定义的变量可以被访问和修改。
### 7.12.2 解决方案
一般来说，在闭包内层定义的变量对于外界来说完全时隔离的。但是，可以通过编写存取函数(accessor function，即getter/setter方法)并将它们作为函数属性暑假到闭包上来提供对内层变量的访问支持。

In [1]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n=', n)
    # Accessor method for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value
    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

In [2]:
f = sample()
f()

n= 0


In [3]:
f.set_n(10)
f()

n= 10


In [4]:
f.get_n()

10

### 7.12.3 讨论
这里主要用到了两个特性使得本节讨论的技术得以成功实施。首先，nonlocal声明使得编写函数来修改内层变量称为可能。其次，函数属性能够将存取函数一直接的方式附加到闭包函数上，它们工作起来很像实例的方法。

对本节提到的技术稍加扩展就可以让闭包模拟成类实例。我们所要作的就是将内层函数拷贝到一个实例的字典中然后将它返回。示例如下：

In [5]:
import sys
class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals
        # Update instance dictionary with callables
        self.__dict__.update((key, value) for key, value in locals.items()
                            if callable(value))
    
    def __len__(self):
        return self.__dict__['__len__']()

# Example use
def Stock():
    items = []
    def push(item):
        items.append(item)
    
    def pop():
        return items.pop()
    
    def __len__():
        return len(items)
    return ClosureInstance()

In [6]:
s = Stock()
s

In [7]:
s.push(10)
s.push(20)
s.push('Hello')
len(s)

3

In [8]:
s.pop()

'Hello'

In [9]:
s.pop()

20

In [10]:
s.pop()

10

In [11]:
# 这份代码运行起来比使用一个普通的类定义要稍微快一些，因为不涉及额外的seld变量
class Stock2:
    def __init__(self):
        self.items = []
    def push(self, item):
        self.items.append(item)
    
    def pop(self):
        return self.items.pop()
    
    def __len__(self):
        return len(self.items)

from timeit import timeit
# Test involving closures
s = Stock()
timeit('s.push(1);s.pop()', 'from __main__ import s')

0.5184259999999767

In [12]:
# Test involving a class
s = Stock2()
timeit('s.push(1);s.pop()', 'from __main__ import s')

0.5304671000000099